In [48]:
import bamboolib as bam #easy to do data manipulation and highly recommend going to https://bamboolib.8080labs.com/ and that will help you follow along
import pandas as pd #data manipulation
import matplotlib.pyplot as plt
import numpy as np

y=pd.read_csv(r'C:\Netflix_Revision.csv')

Data Cleanup & Investigation

In [49]:
y = y.drop(columns=['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'])
y = y.loc[~(y['Episode'].isna())]
y = y.loc[~(y['Gender'].isna())]
y[['Completed']] = y[['Completed']].fillna(y[['Completed']].mode().iloc[0])
y = y.loc[~(y['Time Watched'] >= 75)]
y['User ID'] = y['User ID'].astype('string')
y['Day Of Week'] = pd.to_datetime(y['Day Of Week'], infer_datetime_format=True)
cleaned_column_names = (y.columns
                        .str.strip()
                        .str.replace('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))', r'_\1')
                        .str.lower()
                        .str.replace('[ _-]+', '_')
                        .str.replace('[}{)(><.!?\\\\:;,-]', ''))
y.columns = cleaned_column_names
y = y.rename(columns={'unnamed_2': 'day'})
y['day'] = y['day'].astype('string')
y['show'] = y['show'].astype('string')
y['gender'] = y['gender'].astype('string')
y = pd.get_dummies(y, columns=['gender'], drop_first=True, dummy_na=False)
y = pd.get_dummies(y, columns=['show'], drop_first=True, dummy_na=False)
y = pd.get_dummies(y, columns=['day'], drop_first=True, dummy_na=False)
y

,user_id,day_of_week,season,episode,time_watched,completed,time_of_day,gender_Male,show_Friends,show_Orange Is The New Black,show_Parks & Rec,show_Stranger Things,day_Saturday,day_Sunday,day_Tuesday
0,117752,2019-02-12,2,6.0,51,1.0,1,0,0,0,0,1,0,0,1
1,28615,2019-02-12,2,6.0,2,0.0,0,0,0,0,0,1,0,0,1
2,72226,2019-02-12,2,3.0,5,0.0,1,0,0,0,0,1,0,0,1
3,62565,2019-02-09,2,10.0,59,1.0,1,0,0,0,0,1,1,0,0
4,3494,2019-02-12,2,1.0,33,1.0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028,24988,2019-02-11,4,2.0,25,0.0,1,0,0,0,0,0,0,0,0
3029,15248,2019-02-11,4,2.0,16,0.0,0,1,0,0,0,0,0,0,0
3030,4959,2019-02-09,2,1.0,20,0.0,1,1,0,0,0,0,1,0,0
3031,28654,2019-02-10,5,2.0,12,0.0,1,1,0,0,0,0,0,1,0


Model Preprocessing

In [50]:
from sklearn.model_selection import train_test_split


y
#Split
#x_train, x_test, z_train, z_test = train_test_split(x, z, test_size=0.20)
x = y[['season', 'episode', 'completed', 'time_of_day', 'gender_Male','show_Friends','show_Orange Is The New Black','show_Parks & Rec','show_Stranger Things']]
Y = y[['time_watched']]

x_train, x_test, z_train, z_test = train_test_split(x, Y, test_size=0.20)

Model

In [51]:
import statsmodels.api as sm

lr = sm.add_constant(x_train) 
lr_model = sm.OLS(z_train, lr) 
lr_fit = lr_model.fit()
print(lr_fit.summary())


                            OLS Regression Results                            
Dep. Variable:           time_watched   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.609
Method:                 Least Squares   F-statistic:                     401.8
Date:                Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:38:36   Log-Likelihood:                -8645.7
No. Observations:                2317   AIC:                         1.731e+04
Df Residuals:                    2307   BIC:                         1.737e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

Look At Results

In [53]:
x = y[['completed','show_Orange Is The New Black']]
Y = y[['time_watched']]

x_train, x_test, z_train, z_test = train_test_split(x, Y, test_size=0.20)

lr = sm.add_constant(x_train) 
lr_model = sm.OLS(z_train, lr) 
lr_fit = lr_model.fit()
print(lr_fit.summary())

                            OLS Regression Results                            
Dep. Variable:           time_watched   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.597
Method:                 Least Squares   F-statistic:                     1717.
Date:                Thu, 20 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:47:17   Log-Likelihood:                -8678.3
No. Observations:                2317   AIC:                         1.736e+04
Df Residuals:                    2314   BIC:                         1.738e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           